In [3]:
import pandas as pd
pd.set_option('display.max_columns',False)
import numpy as np
import datetime as dt
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
connection = psycopg2.connect(
    host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com",
    database="mid_term_project",
    user="lhl_student",
    password="lhl_student",
    port=5432)

In [3]:
df_fuel = pd.read_sql_query('SELECT * FROM fuel_comsumption', connection)

In [6]:
df_fuel.to_csv('fuel.csv',index=False)

In [ ]:
# df_flights = pd.read_sql_query('SELECT * FROM flights ORDER BY RANDOM() LIMIT 3000;', connection)
# df_fuel = pd.read_sql_query('SELECT * FROM fuel_comsumption ORDER BY RANDOM() LIMIT 3000;', connection)
# df_passenger = pd.read_sql_query('SELECT * FROM passengers ORDER BY RANDOM() LIMIT 3000;', connection)

In [70]:
df_passenger = pd.read_sql_query("SELECT * FROM passengers WHERE year = 2020 ORDER BY RANDOM() LIMIT 500000;", connection)

In [37]:
df_flights_2019 = pd.read_sql_query("SELECT * FROM flights WHERE date_part('year', TO_DATE(fl_date,'YYYY-MM-DD')) = 2019 ORDER BY RANDOM() LIMIT 300000;", connection)

In [39]:
df_flights_2019.to_csv('flights_2019_300000.csv',index=False)

In [74]:
df_passenger.to_csv('passenger500000.csv',index=False)

In [2]:
df_flights = pd.read_csv('flights_2019_300000.csv')

In [175]:
weather = pd.read_csv('weather.csv')

In [43]:
df_flights = df_flights_2019.copy()

In [3]:
df_flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-06-18,DL,DL_CODESHARE,DL,7408,OO,N433SW,7408,11433,DTW,"Detroit, MI",14150,PLN,"Pellston, MI",2030,2225.0,115.0,13.0,2238.0,2322.0,6.0,2142,2328.0,106.0,0.0,NaN,0.0,N,72.0,63.0,44.0,1.0,243.0,97.0,0.0,0.0,0.0,9.0,NaN,NaN,NaN,NaN
1,2019-07-02,AA,AA_CODESHARE,AA,5780,YV,N912FJ,5780,11298,DFW,"Dallas/Fort Worth, TX",12896,LBB,"Lubbock, TX",845,940.0,55.0,16.0,956.0,1039.0,3.0,956,1042.0,46.0,0.0,NaN,0.0,N,71.0,62.0,43.0,1.0,282.0,0.0,0.0,0.0,0.0,46.0,NaN,NaN,NaN,NaN
2,2019-10-11,UA,UA_CODESHARE,UA,3829,ZW,N421ZW,3829,14685,SAV,"Savannah, GA",13930,ORD,"Chicago, IL",730,759.0,29.0,20.0,819.0,911.0,12.0,904,923.0,19.0,0.0,NaN,0.0,N,154.0,144.0,112.0,1.0,773.0,19.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2019-10-14,AA,AA_CODESHARE,AA,5067,OH,N593NN,5067,11057,CLT,"Charlotte, NC",11423,DSM,"Des Moines, IA",2227,2220.0,-7.0,32.0,2252.0,7.0,4.0,3,11.0,8.0,0.0,NaN,0.0,N,156.0,171.0,135.0,1.0,815.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-10-26,DL,DL,DL,986,DL,N359DN,986,12478,JFK,"New York, NY",14869,SLC,"Salt Lake City, UT",659,655.0,-4.0,14.0,709.0,945.0,7.0,1017,952.0,-25.0,0.0,NaN,0.0,N,318.0,297.0,276.0,1.0,1990.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df_flights['op_unique_carrier'].value_counts()

WN    50366
DL    36485
AA    35417
OO    30776
UA    23215
YX    12310
MQ    12283
B6    11056
OH    10594
AS     9974
9E     9471
YV     8483
NK     7587
EV     5048
F9     4918
PT     4590
QX     4548
ZW     4012
G4     3957
CP     3440
HA     3074
G7     2924
AX     2920
C5     2165
EM      332
KS       55
Name: op_unique_carrier, dtype: int64

In [72]:
weather

,date_time,location,totalSnow_cm,precipMM,visibility,windspeedKmph
0,2019-01-01,ABE,0.0,17.4,9,18
1,2019-01-02,ABE,0.0,0.1,10,12
2,2019-01-03,ABE,0.0,1.4,7,7
3,2019-01-04,ABE,0.0,0.0,9,11
4,2019-01-05,ABE,0.0,25.3,8,11
...,...,...,...,...,...,...
135775,2019-12-27,YUM,0.0,0.5,10,12
135776,2019-12-28,YUM,0.0,0.0,10,8
135777,2019-12-29,YUM,0.0,0.0,10,8
135778,2019-12-30,YUM,0.0,0.0,10,13


features to remove?

comparing flights_test columns to training data columns. test dataset is missing the following columns: dep_time, dep_delay taxi_out wheels_off wheels_on taxi_in arr_time cancelled cancellation_code diverted dup actual_elapsed_time air_time carrier_delay weather_delay nas_delay security_delay late_aircraft_delay first_dep_time total_add_gtime longest_add_gtime no_name

remove: first_dep_time, total_add_gtime, longest_add_gtime, no_name, dup, mkt_unique_carrier, branded_code_share, op_unique_carrier, op_carrier_fl_num, tail_num, origin_airport_id, dest_airport_id, cancelled, cancellation_code, flights

delay categories: carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay. Fill na with 0 as they have no delays

target is arr_delay: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
does it make sense to remove the delay categories for now? Used for the stretch goal. 

What to use for origin and destination classifiers? dest? origin? city name?

need to split origin_city_name/dest_city_name

feature engineering: check correlation matrix or do a pca

In [45]:
df_flights.isnull().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier                 0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                 1051
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                 5613
dep_delay                5613
taxi_out                 5775
wheels_off               5775
wheels_on                5934
taxi_in                  5934
crs_arr_time                0
arr_time                 5934
arr_delay                6648
cancelled                   0
cancellation_code      294189
diverted                    0
dup                         0
crs_elapsed_time            1
actual_elapsed_time      6648
air_time                 6648
flights                     0
distance                    0
carrier_de

In [4]:
#drop flights that were cancelled as they have no delays
df_flights = df_flights[df_flights['cancelled']==0]

In [177]:
#split the city and country for origin and destination
# df_flights[['origin_city','origin_state']] = df_flights['origin_city_name'].str.split(',', expand=True)
# df_flights[['dest_city','dest_state']] = df_flights['dest_city_name'].str.split(',', expand=True)

In [5]:
#drop some columns we dont need. first_dep_time, total_add_gtime, longest_add_gtime, no_name are mainly filled with null values. As we are dealing with delays, any cancellations are not subject to delays. diverted flights also do not make it
# to their original destination
df_flights = df_flights.drop(['first_dep_time','total_add_gtime','longest_add_gtime','no_name','cancelled','cancellation_code','diverted'],axis=1)

In [6]:
df_flights['carrier_delay'].isna().sum()/len(df_flights)*100

80.81913327826669

In [7]:
#drop the 5 delay columns for the time being. A large percent of these values are na values
df_flights = df_flights.drop(['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay'],axis=1)

In [11]:
#Fillna for the 5 delay columns, as the nan values appeared when there were no delays at all
# df_flights = df_flights.fillna({'carrier_delay':0,'weather_delay':0,'nas_delay':0,'security_delay':0,'late_aircraft_delay':0})

In [8]:
#drop na rows in following columns. We have a big enough dataset to ignore these.
df_flights = df_flights.dropna(subset=['tail_num','dep_time','dep_delay','taxi_out','wheels_off','wheels_on','taxi_in','arr_time','arr_delay','actual_elapsed_time','air_time'])

In [9]:
#change all times in the hhmm format to be integers for consistency
df_flights[['dep_time','wheels_off','wheels_on','arr_time']] = df_flights[['dep_time','wheels_off','wheels_on','arr_time']].astype(int)

In [10]:
# won't be using the flight number in our prediction, as well as some of the identifiers. Drop some related carrier identifiers, non-split city names. only keeping op_unique_carrier as our identifier
df_flights = df_flights.drop(['mkt_unique_carrier','branded_code_share','mkt_carrier','tail_num','dup','origin_city_name','dest_city_name','mkt_carrier_fl_num','origin_airport_id','dest_airport_id','mkt_carrier_fl_num','op_carrier_fl_num','flights'],axis=1)

In [11]:
#is dep_time and arr_time necessary? dep_delay and arr_delay tells us what we already need.
df_flights = df_flights.drop(['dep_time','arr_time'],axis=1)

In [12]:
#drop columns not in testing dataset
df_flights = df_flights.drop(['dep_delay','taxi_out','wheels_off','wheels_on','taxi_in','actual_elapsed_time','air_time'],axis=1)

In [186]:
# drop origin state and destination state as they are not useful to us
# df_flights = df_flights.drop(['origin_state','dest_state'],axis=1)

In [13]:
#round crs times to the hour, rather than hhmm
df_flights['crs_dep_time'] = df_flights['crs_dep_time']//100
df_flights['crs_arr_time'] = df_flights['crs_arr_time']//100

In [14]:
#seperate fl_date to month and day of the week. monday is 0,tuesday = 1, wednesday = 2, thursday = 3, friday = 4, saturday = 5, sunday = 6
df_flights['fl_date'] = pd.to_datetime(df_flights['fl_date'])
df_flights['month'] = df_flights['fl_date'].dt.month
df_flights['weekday']=df_flights['fl_date'].dt.weekday

In [ ]:
#should pick either origin/dest or the origin/destination city
# df_flights = df_flights.drop(['fl_date','origin','dest'],axis=1)

In [28]:
# was unable to get weather information for this location. Seems like there is only 1 location as well, so just drop to save some hassle.
df_flights = df_flights[df_flights['origin']!='XWA']
df_flights = df_flights[df_flights['dest']!='XWA']

In [36]:
df_flights.head()

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,month,weekday
0,2019-06-18,OO,DTW,PLN,20,21,106.0,72.0,243.0,6,1
1,2019-07-02,YV,DFW,LBB,8,9,46.0,71.0,282.0,7,1
2,2019-10-11,ZW,SAV,ORD,7,9,19.0,154.0,773.0,10,4
3,2019-10-14,OH,CLT,DSM,22,0,8.0,156.0,815.0,10,0
4,2019-10-26,DL,JFK,SLC,6,10,-25.0,318.0,1990.0,10,5


In [20]:
df_flights['origin'].nunique()

371

In [176]:
weather.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2019-01-01,11,4,0.0,8.7,3,26,04:05 AM,02:53 PM,08:25 AM,05:45 PM,5,6,8,6,32,33,80,17.4,1013,11,9,263,18,ABE
1,2019-01-02,4,0,0.0,3.5,1,19,05:08 AM,03:29 PM,08:25 AM,05:46 PM,-2,0,3,0,9,60,74,0.1,1023,4,10,180,12,ABE
2,2019-01-03,6,0,0.0,8.7,1,11,06:08 AM,04:07 PM,08:26 AM,05:47 PM,0,1,3,1,17,34,79,1.4,1016,6,7,248,7,ABE
3,2019-01-04,6,0,0.0,7.0,1,4,07:06 AM,04:51 PM,08:26 AM,05:48 PM,0,0,3,0,13,31,81,0.0,1015,6,9,222,11,ABE
4,2019-01-05,6,2,0.0,5.2,1,0,08:00 AM,05:39 PM,08:26 AM,05:49 PM,3,1,4,1,20,68,92,25.3,1003,6,8,169,11,ABE


In [177]:
#choosing weather data to use
weather = weather[['date_time','location','totalSnow_cm','precipMM','visibility','windspeedKmph']]

In [178]:
weather.head()

,date_time,location,totalSnow_cm,precipMM,visibility,windspeedKmph
0,2019-01-01,ABE,0.0,17.4,9,18
1,2019-01-02,ABE,0.0,0.1,10,12
2,2019-01-03,ABE,0.0,1.4,7,7
3,2019-01-04,ABE,0.0,0.0,9,11
4,2019-01-05,ABE,0.0,25.3,8,11


In [44]:
df_flights.head()

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,month,weekday
0,2019-06-18,OO,DTW,PLN,20,21,106.0,72.0,243.0,6,1
1,2019-07-02,YV,DFW,LBB,8,9,46.0,71.0,282.0,7,1
2,2019-10-11,ZW,SAV,ORD,7,9,19.0,154.0,773.0,10,4
3,2019-10-14,OH,CLT,DSM,22,0,8.0,156.0,815.0,10,0
4,2019-10-26,DL,JFK,SLC,6,10,-25.0,318.0,1990.0,10,5


In [244]:
# using origin and dest, drop origin_city and dest_city
# df_flights = df_flights.drop(['origin_city','dest_city'],axis=1)

In [45]:
df_flights2 = df_flights.copy()

In [46]:
df_flights2

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,month,weekday
0,2019-06-18,OO,DTW,PLN,20,21,106.0,72.0,243.0,6,1
1,2019-07-02,YV,DFW,LBB,8,9,46.0,71.0,282.0,7,1
2,2019-10-11,ZW,SAV,ORD,7,9,19.0,154.0,773.0,10,4
3,2019-10-14,OH,CLT,DSM,22,0,8.0,156.0,815.0,10,0
4,2019-10-26,DL,JFK,SLC,6,10,-25.0,318.0,1990.0,10,5
...,...,...,...,...,...,...,...,...,...,...,...
299994,2019-04-25,OO,SBN,MSP,7,7,-8.0,115.0,411.0,4,3
299995,2019-03-23,EV,TUL,ORD,6,8,4.0,127.0,585.0,3,5
299996,2019-01-21,NK,BOS,RSW,6,10,19.0,227.0,1249.0,1,0
299998,2019-01-30,WN,DAL,IND,15,18,-16.0,120.0,755.0,1,2


In [54]:
def make_bins(df):
    '''
    Bin departure and arrival times to morning, afternoon, evening, night
    Morning: 12am-5am
    Afternoon: 6am-11am
    Evening: 12pm-5pm
    Night: 6pm-11pm
    '''
    label_names = ['Morning','Afternoon','Evening','Night']
    cut_points = [0,5,11,17,23]
    df_flights2["crs_dep_bin"] = pd.cut(df_flights2["crs_dep_time"], cut_points, labels=label_names,include_lowest=True)
    df_flights2["crs_arr_bin"] = pd.cut(df_flights2["crs_arr_time"], cut_points, labels=label_names,include_lowest=True)
    return df

In [55]:
make_bins(df_flights2)

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,month,weekday,crs_dep_bin,crs_arr_bin
0,2019-06-18,OO,DTW,PLN,20,21,106.0,72.0,243.0,6,1,Night,Night
1,2019-07-02,YV,DFW,LBB,8,9,46.0,71.0,282.0,7,1,Afternoon,Afternoon
2,2019-10-11,ZW,SAV,ORD,7,9,19.0,154.0,773.0,10,4,Afternoon,Afternoon
3,2019-10-14,OH,CLT,DSM,22,0,8.0,156.0,815.0,10,0,Night,Morning
4,2019-10-26,DL,JFK,SLC,6,10,-25.0,318.0,1990.0,10,5,Afternoon,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
299994,2019-04-25,OO,SBN,MSP,7,7,-8.0,115.0,411.0,4,3,Afternoon,Afternoon
299995,2019-03-23,EV,TUL,ORD,6,8,4.0,127.0,585.0,3,5,Afternoon,Afternoon
299996,2019-01-21,NK,BOS,RSW,6,10,19.0,227.0,1249.0,1,0,Afternoon,Afternoon
299998,2019-01-30,WN,DAL,IND,15,18,-16.0,120.0,755.0,1,2,Evening,Night


In [179]:
weather['date_time'] = pd.to_datetime(weather['date_time'])

In [57]:
#merge weather data to dataset
flights_weather = pd.merge(df_flights2,weather, how='left',left_on=['fl_date','origin'], right_on=['date_time','location'])

In [58]:
flights_weather

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,month,weekday,crs_dep_bin,crs_arr_bin,date_time,location,totalSnow_cm,precipMM,visibility,windspeedKmph
0,2019-06-18,OO,DTW,PLN,20,21,106.0,72.0,243.0,6,1,Night,Night,2019-06-18,DTW,0.0,0.4,10,8
1,2019-07-02,YV,DFW,LBB,8,9,46.0,71.0,282.0,7,1,Afternoon,Afternoon,2019-07-02,DFW,0.0,6.5,9,18
2,2019-10-11,ZW,SAV,ORD,7,9,19.0,154.0,773.0,10,4,Afternoon,Afternoon,2019-10-11,SAV,0.0,0.1,10,8
3,2019-10-14,OH,CLT,DSM,22,0,8.0,156.0,815.0,10,0,Night,Morning,2019-10-14,CLT,0.0,0.7,8,5
4,2019-10-26,DL,JFK,SLC,6,10,-25.0,318.0,1990.0,10,5,Afternoon,Afternoon,2019-10-26,JFK,0.0,0.1,10,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293318,2019-04-25,OO,SBN,MSP,7,7,-8.0,115.0,411.0,4,3,Afternoon,Afternoon,2019-04-25,SBN,0.0,3.3,10,7
293319,2019-03-23,EV,TUL,ORD,6,8,4.0,127.0,585.0,3,5,Afternoon,Afternoon,2019-03-23,TUL,0.0,21.7,9,20
293320,2019-01-21,NK,BOS,RSW,6,10,19.0,227.0,1249.0,1,0,Afternoon,Afternoon,2019-01-21,BOS,0.1,0.2,9,30
293321,2019-01-30,WN,DAL,IND,15,18,-16.0,120.0,755.0,1,2,Evening,Night,2019-01-30,DAL,0.0,0.0,10,10


In [59]:
#save the cleaned data to csv
flights_weather.to_csv('cleaned_flights_300000sample1.csv',index=False)

create box plot, remove outliers
make a for loop to get rid of 95% percentile

remove outliers, try for 3 standard deviations from the mean rather than removing points outside the 95th percentile

In [202]:
df_flights = pd.read_csv('cleaned_flights_300000sample1.csv')

In [15]:
std = df_flights['arr_delay'].std()
mean = df_flights['arr_delay'].mean()
outlier_cutoff = mean + 3*std

In [20]:
df_flights = df_flights[df_flights['arr_delay']<outlier_cutoff]

In [21]:
df_flights.to_csv('cleaned_flights_300000sample1.csv',index=False)

### Cleaning test data

In [25]:
df_test = pd.read_sql_query('SELECT * FROM flights_test', connection)

In [4]:
df_test = pd.read_csv('flights_test.csv')

In [5]:
df_test = df_test[df_test['fl_date'] < '2020-01-08']

In [6]:
#drop some features that are not needed
df_test = df_test.drop(['mkt_unique_carrier','branded_code_share','mkt_carrier','mkt_carrier_fl_num','tail_num','origin_airport_id','origin_city_name','dest_airport_id','dest_city_name','dup','flights','op_carrier_fl_num'],axis=1)

In [7]:
#seperate fl_date to month and day of the week. monday is 0,tuesday = 1, wednesday = 2, thursday = 3, friday = 4, saturday = 5, sunday = 6
df_test['fl_date'] = pd.to_datetime(df_test['fl_date'])
df_test['month'] = df_test['fl_date'].dt.month
df_test['weekday'] = df_test['fl_date'].dt.weekday

In [8]:
#round crs times to the hour, rather than hhmm
df_test['crs_dep_time'] = df_test['crs_dep_time']//100
df_test['crs_arr_time'] = df_test['crs_arr_time']//100

In [9]:
df_test.head()

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,month,weekday
0,2020-01-01,WN,ONT,SFO,18,19,95,363,1,2
1,2020-01-01,WN,ONT,SFO,11,13,90,363,1,2
2,2020-01-01,WN,ONT,SJC,20,21,70,333,1,2
3,2020-01-01,WN,ONT,SJC,13,14,75,333,1,2
4,2020-01-01,WN,ONT,SJC,9,10,80,333,1,2


In [44]:
def make_bins_test(df):
    '''
    Bin departure and arrival times to morning, afternoon, evening, night
    Morning: 12am-5am
    Afternoon: 6am-11am
    Evening: 12pm-5pm
    Night: 6pm-11pm
    '''
    label_names = ['Morning','Afternoon','Evening','Night']
    cut_points = [0,5,11,17,23]
    df_test["crs_dep_bin"] = pd.cut(df_test["crs_dep_time"], cut_points, labels=label_names,include_lowest=True)
    df_test["crs_arr_bin"] = pd.cut(df_test["crs_arr_time"], cut_points, labels=label_names,include_lowest=True)
    return df

In [45]:
make_bins_test(df_test)

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,month,weekday,crs_dep_bin,crs_arr_bin
0,2020-01-01,WN,ONT,SFO,18,19,95,363,1,2,Night,Night
1,2020-01-01,WN,ONT,SFO,11,13,90,363,1,2,Afternoon,Evening
2,2020-01-01,WN,ONT,SJC,20,21,70,333,1,2,Night,Night
3,2020-01-01,WN,ONT,SJC,13,14,75,333,1,2,Evening,Evening
4,2020-01-01,WN,ONT,SJC,9,10,80,333,1,2,Afternoon,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...
150618,2020-01-07,9E,DTW,JFK,17,19,117,509,1,1,Evening,Night
150619,2020-01-07,9E,GSP,LGA,6,7,119,610,1,1,Afternoon,Afternoon
150620,2020-01-07,9E,ATL,XNA,17,18,121,589,1,1,Evening,Night
150621,2020-01-07,9E,XNA,ATL,18,21,114,589,1,1,Night,Night


In [10]:
df_test_weather = pd.read_csv('jan2020_weather.csv')

In [11]:
df_test_weather['date_time'] = pd.to_datetime(df_test_weather['date_time'])

In [12]:
#choosing weather data to use
df_test_weather = df_test_weather[['date_time','location','totalSnow_cm','precipMM','visibility','windspeedKmph']]

In [13]:
#merge test weather data to dataset
df_test_weather = pd.merge(df_test,df_test_weather, how='left',left_on=['fl_date','origin'], right_on=['date_time','location'])

In [59]:
# df_test_weather.to_csv('test_sample_cleaned.csv',index=False)

In [7]:
df_test = pd.read_csv('test_sample_cleaned.csv')

In [10]:
df_test = df_test.drop(['date_time','location'],axis=1)

In [1]:
df_test

NameError: name 'df_test' is not defined

No 2020 passenger details, use last years numbers?

In [26]:
df_passenger = df_passenger.drop(['airline_id','carrier_name','origin_city_name','dest_city_name','region','carrier','origin_airport_id','origin_city_market_id','dest_airport_id','dest_city_market_id'],axis=1)

In [27]:
df_passenger = df_passenger.drop(['carrier_group','carrier_group_new','origin_country_name','dest_country','dest_country_name','origin_country','data_source','aircraft_group','aircraft_type','aircraft_config','distance_group','class',],axis=1)

In [28]:
df_passenger = df_passenger.drop(['departures_scheduled'],axis=1)

In [50]:
df_passenger.head()

,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,unique_carrier,unique_carrier_name,origin,dest,year,month
0,4.0,60800.0,304.0,286.0,0.0,0.0,1114.0,717.0,589.0,OO,SkyWest Airlines Inc.,YVR,DEN,2019,10
1,5.0,162087.0,0.0,0.0,158205.0,0.0,463.0,472.0,386.0,5X,United Parcel Service,RIC,SDF,2019,3
2,45.0,1638000.0,8190.0,7545.0,0.0,0.0,1119.0,8184.0,6895.0,NK,Spirit Air Lines,FLL,DFW,2019,6
3,9.0,196227.0,684.0,501.0,0.0,0.0,476.0,936.0,739.0,YX,Republic Airline,IAD,IND,2019,11
4,1.0,38300.0,150.0,147.0,0.0,176.0,1849.0,260.0,215.0,AA,American Airlines Inc.,PHX,MCO,2019,6


In [30]:
#group the flights together by their carrier, month, origin and destination
grouped_pass = df_passenger.groupby(['unique_carrier','origin','dest','month'],axis=0, as_index=False).sum()

In [31]:
#get the average number of passengers for each flight route
grouped_pass['avg_passengers'] = round(grouped_pass['passengers']/grouped_pass['departures_performed'])

In [32]:
# percent capacity of filled seats
grouped_pass['avg_percent_capacity'] = round((grouped_pass['passengers']/grouped_pass['seats']),2)

In [33]:
grouped_pass

,unique_carrier,origin,dest,month,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,year,avg_passengers,avg_percent_capacity
0,02Q,ACY,DUB,10,1.0,26543.0,80.0,58.0,0.0,0.0,3265.0,0.0,0.0,2019,58.0,0.72
1,02Q,ANC,KIX,5,1.0,26543.0,80.0,68.0,0.0,0.0,3680.0,0.0,0.0,2019,68.0,0.85
2,02Q,BFI,KIX,5,1.0,26543.0,80.0,68.0,0.0,0.0,5037.0,0.0,0.0,2019,68.0,0.85
3,02Q,BOS,STN,6,1.0,26543.0,80.0,0.0,0.0,0.0,3285.0,0.0,0.0,2019,0.0,0.00
4,02Q,CMH,EDI,11,1.0,55000.0,265.0,265.0,0.0,0.0,3629.0,0.0,0.0,2019,265.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304862,ZX,YYZ,STL,7,92.0,1118905.0,4600.0,4015.0,0.0,0.0,654.0,0.0,0.0,2019,44.0,0.87
304863,ZX,YYZ,STL,8,93.0,1131066.0,4650.0,3642.0,0.0,0.0,654.0,0.0,0.0,2019,39.0,0.78
304864,ZX,YYZ,STL,9,116.0,1410792.0,5800.0,4279.0,0.0,0.0,654.0,0.0,0.0,2019,37.0,0.74
304865,ZX,YYZ,STL,10,118.0,1435115.0,5900.0,4450.0,0.0,0.0,654.0,0.0,0.0,2019,38.0,0.75


In [34]:
#merge passenger data with test data 
df_test_passengers = pd.merge(df_test,grouped_pass, how='left',left_on=['op_unique_carrier','origin','dest','month'], right_on=['unique_carrier','origin','dest','month'])

In [48]:
df_test_passengers

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_x,month,weekday,crs_dep_bin,crs_arr_bin,totalSnow_cm,precipMM,visibility,windspeedKmph,avg_passengers,avg_percent_capacity
0,2020-01-01,WN,ONT,SFO,18,19,95,363,1,2,Night,Night,0.0,0.0,10,4,NaN,NaN
1,2020-01-01,WN,ONT,SFO,11,13,90,363,1,2,Afternoon,Evening,0.0,0.0,10,4,NaN,NaN
2,2020-01-01,WN,ONT,SJC,20,21,70,333,1,2,Night,Night,0.0,0.0,10,4,104.0,0.72
3,2020-01-01,WN,ONT,SJC,13,14,75,333,1,2,Evening,Evening,0.0,0.0,10,4,104.0,0.72
4,2020-01-01,WN,ONT,SJC,9,10,80,333,1,2,Afternoon,Afternoon,0.0,0.0,10,4,104.0,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150618,2020-01-07,9E,DTW,JFK,17,19,117,509,1,1,Evening,Night,0.0,0.0,10,16,57.0,0.75
150619,2020-01-07,9E,GSP,LGA,6,7,119,610,1,1,Afternoon,Afternoon,0.0,4.2,10,10,69.0,0.91
150620,2020-01-07,9E,ATL,XNA,17,18,121,589,1,1,Evening,Night,0.0,0.8,10,18,53.0,0.72
150621,2020-01-07,9E,XNA,ATL,18,21,114,589,1,1,Night,Night,0.0,0.0,10,13,50.0,0.68


In [43]:
df_test_passengers = df_test_passengers.drop(['unique_carrier','departures_performed','payload','seats','passengers','freight','mail','distance_y','ramp_to_ramp','air_time','year'],axis=1)

In [44]:
df_test_passengers.head()

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_x,month,weekday,crs_dep_bin,crs_arr_bin,totalSnow_cm,precipMM,visibility,windspeedKmph,avg_passengers,avg_percent_capacity
0,2020-01-01,WN,ONT,SFO,18,19,95,363,1,2,Night,Night,0.0,0.0,10,4,NaN,NaN
1,2020-01-01,WN,ONT,SFO,11,13,90,363,1,2,Afternoon,Evening,0.0,0.0,10,4,NaN,NaN
2,2020-01-01,WN,ONT,SJC,20,21,70,333,1,2,Night,Night,0.0,0.0,10,4,104.0,0.72
3,2020-01-01,WN,ONT,SJC,13,14,75,333,1,2,Evening,Evening,0.0,0.0,10,4,104.0,0.72
4,2020-01-01,WN,ONT,SJC,9,10,80,333,1,2,Afternoon,Afternoon,0.0,0.0,10,4,104.0,0.72


In [51]:
df_test_passengers['avg_passengers'] = df_test_passengers['avg_passengers'].fillna(df_test_passengers['avg_passengers'].mean())

In [53]:
df_test_passengers['avg_percent_capacity'] = df_test_passengers['avg_percent_capacity'].fillna(df_test_passengers['avg_percent_capacity'].mean())

In [57]:
df_test_passengers.to_csv('test_sample_cleaned.csv')

## Explore passengers data

In [24]:
df_passenger = pd.read_csv('passenger500000.csv')

In [ ]:
df_passenger.isnull().sum()

In [16]:
df_passenger = df_passenger.drop(['carrier_group','carrier_group_new','origin_country_name','dest_country','dest_country_name','origin_country','data_source','aircraft_group','aircraft_type','aircraft_config','distance_group','class',],axis=1)

In [17]:
df_passenger = df_passenger.drop(['airline_id','carrier_name','origin_city_name','dest_city_name','region','carrier','origin_airport_id','origin_city_market_id','dest_airport_id','dest_city_market_id'],axis=1)

In [18]:
df_passenger = df_passenger.drop(['departures_scheduled'],axis=1)

In [144]:
df_passenger.head()

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,unique_carrier,unique_carrier_name,origin,dest,year,month
0,4.0,4.0,60800.0,304.0,286.0,0.0,0.0,1114.0,717.0,589.0,OO,SkyWest Airlines Inc.,YVR,DEN,2019,10
1,5.0,5.0,162087.0,0.0,0.0,158205.0,0.0,463.0,472.0,386.0,5X,United Parcel Service,RIC,SDF,2019,3
2,45.0,45.0,1638000.0,8190.0,7545.0,0.0,0.0,1119.0,8184.0,6895.0,NK,Spirit Air Lines,FLL,DFW,2019,6
3,9.0,9.0,196227.0,684.0,501.0,0.0,0.0,476.0,936.0,739.0,YX,Republic Airline,IAD,IND,2019,11
4,1.0,1.0,38300.0,150.0,147.0,0.0,176.0,1849.0,260.0,215.0,AA,American Airlines Inc.,PHX,MCO,2019,6


In [151]:
#group the flights together by their carrier, month, origin and destination
grouped_pass = df_passenger.groupby(['unique_carrier','origin','dest','month'],axis=0, as_index=False).sum()

In [152]:
#get the average number of passengers for each flight route
grouped_pass['avg_passengers'] = round(grouped_pass['passengers']/grouped_pass['departures_performed'])

In [153]:
# percent capacity of filled seats
grouped_pass['avg_percent_capacity'] = round((grouped_pass['passengers']/grouped_pass['seats']),2)

In [154]:
grouped_pass

,unique_carrier,origin,dest,month,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,year,avg_passengers,avg_percent_capacity
0,02Q,ACY,DUB,10,1.0,26543.0,80.0,58.0,0.0,0.0,3265.0,0.0,0.0,2019,58.0,0.72
1,02Q,ANC,KIX,5,1.0,26543.0,80.0,68.0,0.0,0.0,3680.0,0.0,0.0,2019,68.0,0.85
2,02Q,BFI,KIX,5,1.0,26543.0,80.0,68.0,0.0,0.0,5037.0,0.0,0.0,2019,68.0,0.85
3,02Q,BOS,STN,6,1.0,26543.0,80.0,0.0,0.0,0.0,3285.0,0.0,0.0,2019,0.0,0.00
4,02Q,CMH,EDI,11,1.0,55000.0,265.0,265.0,0.0,0.0,3629.0,0.0,0.0,2019,265.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304862,ZX,YYZ,STL,7,92.0,1118905.0,4600.0,4015.0,0.0,0.0,654.0,0.0,0.0,2019,44.0,0.87
304863,ZX,YYZ,STL,8,93.0,1131066.0,4650.0,3642.0,0.0,0.0,654.0,0.0,0.0,2019,39.0,0.78
304864,ZX,YYZ,STL,9,116.0,1410792.0,5800.0,4279.0,0.0,0.0,654.0,0.0,0.0,2019,37.0,0.74
304865,ZX,YYZ,STL,10,118.0,1435115.0,5900.0,4450.0,0.0,0.0,654.0,0.0,0.0,2019,38.0,0.75


### Merge passengers data with flights data

In [205]:
df_flights2 = pd.read_csv('cleaned_flights_300000sample1.csv')

In [207]:
#merge passenger data with flights data 
df_flights_passengers = pd.merge(df_flights2,grouped_pass, how='left',left_on=['op_unique_carrier','origin','dest','month'], right_on=['unique_carrier','origin','dest','month'])

In [211]:
df_flights_passengers = df_flights_passengers.drop(['distance_y','year','date_time','location','air_time','unique_carrier'],axis=1)

In [214]:
df_flights_passengers.rename(columns={'distance_x':'distance'},inplace=True)

In [224]:
df_flights_passengers

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,month,weekday,crs_dep_bin,crs_arr_bin,totalSnow_cm,precipMM,visibility,windspeedKmph,departures_performed,payload,seats,passengers,freight,mail,ramp_to_ramp,avg_passengers,avg_percent_capacity
0,2019-06-18,OO,DTW,PLN,20,21,106.0,72.0,243.0,6,1,Night,Night,0.0,0.4,10,8,100.0,1238000.0,5000.0,3868.0,0.0,0.0,6429.0,39.0,0.77
1,2019-07-02,YV,DFW,LBB,8,9,46.0,71.0,282.0,7,1,Afternoon,Afternoon,0.0,6.5,9,18,169.0,3415490.0,13150.0,9361.0,1346.0,1260.0,10863.0,55.0,0.71
2,2019-10-11,ZW,SAV,ORD,7,9,19.0,154.0,773.0,10,4,Afternoon,Afternoon,0.0,0.1,10,8,26.0,626600.0,1300.0,1058.0,0.0,0.0,3796.0,41.0,0.81
3,2019-10-14,OH,CLT,DSM,22,0,8.0,156.0,815.0,10,0,Night,Morning,0.0,0.7,8,5,92.0,1746550.0,6343.0,4891.0,42.0,0.0,14307.0,53.0,0.77
4,2019-10-26,DL,JFK,SLC,6,10,-25.0,318.0,1990.0,10,5,Afternoon,Afternoon,0.0,0.1,10,14,115.0,5919710.0,20844.0,18457.0,49708.0,93985.0,35621.0,160.0,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288587,2019-04-25,OO,SBN,MSP,7,7,-8.0,115.0,411.0,4,3,Afternoon,Afternoon,0.0,3.3,10,7,53.0,642000.0,2650.0,2265.0,0.0,0.0,5089.0,43.0,0.85
288588,2019-03-23,EV,TUL,ORD,6,8,4.0,127.0,585.0,3,5,Afternoon,Afternoon,0.0,21.7,9,20,35.0,378000.0,1750.0,1475.0,0.0,0.0,4086.0,42.0,0.84
288589,2019-01-21,NK,BOS,RSW,6,10,19.0,227.0,1249.0,1,0,Afternoon,Afternoon,0.1,0.2,9,30,58.0,2111200.0,10556.0,6921.0,0.0,0.0,12557.0,119.0,0.66
288590,2019-01-30,WN,DAL,IND,15,18,-16.0,120.0,755.0,1,2,Evening,Night,0.0,0.0,10,10,35.0,1387000.0,5645.0,4253.0,7052.0,0.0,3929.0,122.0,0.75


In [219]:
df_flights_passengers['fl_date'].dtype

dtype('O')

In [220]:
df_flights_passengers['fl_date'] = pd.to_datetime(df_flights_passengers['fl_date'])

In [227]:
df_flights_passengers.isnull().sum()

fl_date                  0
op_unique_carrier        0
origin                   0
dest                     0
crs_dep_time             0
crs_arr_time             0
arr_delay                0
crs_elapsed_time         0
distance                 0
month                    0
weekday                  0
crs_dep_bin              0
crs_arr_bin              6
totalSnow_cm             0
precipMM                 0
visibility               0
windspeedKmph            0
departures_performed    28
payload                 28
seats                   28
passengers              28
freight                 28
mail                    28
ramp_to_ramp            28
avg_passengers          28
avg_percent_capacity    28
dtype: int64

In [228]:
df_flights_passengers = df_flights_passengers.dropna()

In [229]:
df_flights_passengers.to_csv('cleaned_flights_sample2.csv',index=False)